<a href="https://colab.research.google.com/github/carloshvmoraes/ISGDTPDN/blob/main/ISGD_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrating Smart Grid Devices into the Traditional Protection of Distribution Networks

## Install Python Libraries

In [7]:
!pip install pandapower

## Download Source ISGDTPDN

In [8]:
!wget https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/src/ISGDTPDN.py -O ISGDTPDN.py
!wget https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/examples/CWB_p64.py -O CWB_p64.py

--2022-03-03 21:37:53--  https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/src/ISGDTPDN.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2509 (2.5K) [text/plain]
Saving to: ‘ISGDTPDN.py’

ISGDTPDN.py         100%[===================>]   2.45K  --.-KB/s    in 0s      

2022-03-03 21:37:53 (29.7 MB/s) - ‘ISGDTPDN.py’ saved [2509/2509]

--2022-03-03 21:37:53--  https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/examples/CWB_p64.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5433 (5.3K) [text/plain]
Savi

## Example - CWB Bus 64

<img src="https://github.com/carloshvmoraes/ISGDTPDN/raw/main/examples/CWB_p64.png" width="500" />

## Basic Use

In [9]:
from ISGDTPDN import CurrSaved, CurrSavedNew, FaultDetect
from CWB_p64 import new_network
import pandapower as pp
import random
import pandas as pd
import IPython

random.seed(42)

# basic settings for 2 scenarios
scenarios = [
            {'sw_meters':[64], 'LoadRange':0.05},
            {'sw_meters':[64, 68], 'LoadRange':0.05},
            ]

for setting in scenarios:
  name_scenario = '-'.join(map(str,setting['sw_meters']))
  display(IPython.display.HTML(f'<h1>Scenario {name_scenario}</h1>'))

  filename = name_scenario + '_' + str(setting['LoadRange']) + '_single'
  
  # create distribuition net
  net = new_network(medidores=setting['sw_meters'])
  
  # load current before changes
  pp.runpp(net)
  I_Load = net.res_bus['p_mw'] * 72.46376811594203 / net.res_bus['vm_pu']
  
  # Fault Pattern Currents
  I_saved = CurrSaved(net)

  # load change by Range
  net.load['p_mw'] = [random.uniform(1-setting['LoadRange'], 1+setting['LoadRange']) * r['p_mw'] for _, r in net.load.iterrows()]

  # load current after changes
  pp.runpp(net)
  I_Load_new = net.res_bus['p_mw'] * 72.46376811594203 / net.res_bus['vm_pu']
  currLoad = pd.concat([I_Load, I_Load_new], axis=1, keys=['BusLoad O', 'BusLoad PF']).sort_index()
  currLoad.index.name = 'Bus'
  display(IPython.display.HTML(f'<h2>Load Currents</h2>'))
  display(currLoad)
  currLoad.to_csv(filename + '_iLoad.csv')

  # Fault Pattern correction
  I_saved_new = CurrSavedNew(net, I_saved)

  solutions = []
  for sw_id, r in net.switch.iterrows():
    # create fault
    net.switch.at[sw_id, 'closed'] = False
    pp.runpp(net, neglect_open_switch_branches=True)
    # remove fault
    net.switch.at[sw_id, 'closed'] = True
    
    # search fault
    fault_best = FaultDetect(net, I_saved_new)
    # best solution
    if sw_id in fault_best:
      index = fault_best.index(sw_id)
      res = {'fault':sw_id, 'choices': fault_best, 'position':(index+1)}
      solutions.append(res)

  dfsol = pd.DataFrame(data=solutions)
  dfsol = dfsol.set_index('fault').sort_index()
  dfsol.to_csv(filename + '_solutions.csv')
  display(IPython.display.HTML(f'<h2>Solutions</h2>'))
  display(dfsol)

,BusLoad O,BusLoad PF
Bus,,
0,-77.500083,-77.418436
64,5.067339,5.167688
65,1.259279,1.239151
66,2.835890,2.738065
67,2.522072,2.637201
68,5.786033,5.690876
69,5.420742,5.199264
70,6.796136,6.520998
71,1.675464,1.733415


,choices,position
fault,,
64,"[64, 65, 66, 67, 68, 69, 82, 73, 78, 70, 74, 8...",1
65,"[65, 66, 67, 64, 68, 69, 82, 73, 78, 70, 74, 8...",1
66,"[66, 67, 68, 65, 69, 64, 82, 73, 78, 70, 74, 8...",1
67,"[67, 68, 66, 65, 69, 82, 73, 78, 64, 70, 74, 8...",1
68,"[68, 67, 66, 69, 65, 82, 73, 78, 70, 74, 80, 8...",1
69,"[69, 82, 73, 78, 70, 74, 80, 68, 84, 81, 79, 7...",1
70,"[74, 70, 78, 80, 73, 84, 81, 79, 75, 72, 82, 7...",2
71,"[71, 76, 83, 77, 72, 75, 79, 81, 84, 80, 74, 7...",1
72,"[72, 75, 79, 81, 77, 84, 71, 76, 83, 80, 74, 7...",1


,BusLoad O,BusLoad PF
Bus,,
0,-79.488457,-79.589138
64,2.512049,2.558455
65,2.912761,3.012662
66,1.610211,1.654685
67,2.583880,2.513926
68,1.759721,1.677440
69,2.891253,2.837933
70,5.171970,5.051784
71,3.448443,3.348698


,choices,position
fault,,
64,"[64, 69, 65, 73, 74, 70, 66, 75, 67, 78, 76, 7...",1
65,"[65, 69, 66, 67, 68, 64, 73, 74, 70, 75, 78, 7...",1
66,"[66, 67, 68, 69, 65, 73, 74, 82, 70, 64, 75, 7...",1
67,"[67, 66, 68, 69, 65, 73, 74, 82, 70, 75, 64, 7...",1
68,"[68, 67, 66, 69, 65, 73, 82, 74, 70, 75, 78, 7...",1
69,"[69, 73, 82, 68, 74, 67, 66, 70, 75, 78, 79, 8...",1
70,"[70, 75, 78, 79, 80, 76, 77, 83, 84, 72, 71, 8...",1
71,"[81, 71, 72, 84, 77, 83, 76, 80, 79, 78, 75, 7...",2
72,"[72, 71, 81, 84, 77, 83, 76, 80, 79, 78, 75, 7...",1
